In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define Pauli matrices
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)
sigma = [sigma_x, sigma_y, sigma_z]  # List of Pauli matrices [x, y, z]

def define_parameters(N=4):
    """
    Define PT-symmetric parameters with real eigenvalues
    """
    # PT-symmetric B field components
    gamma = 1j * 0.5  # Imaginary Bx (small magnitude for unbroken phase)
    lambda_ = 1j * 0.5  # Imaginary By
    bz = 1.0  # Real Bz

    # Coupling strength
    g = 0.1 # Reduced to ensure unbroken PT phase

    # Parameters for B and Gamma
    alpha_x = 1.0
    alpha_y = 1.0
    beta_x = 0.5
    beta_y = 0.5

    # Distinct epsilon values (real)
    epsilon = np.array([0.1, 0.3, 0.5, 0.7], dtype=complex)

    # Calculate B vectors
    B_vectors = np.zeros((N, 3), dtype=complex)
    for i in range(1, N+1):  # Spin indices now run from 1 to N
        B_vectors[i-1, 0] = gamma / np.sqrt(alpha_x * epsilon[i-1] + beta_x)  # Bx (imaginary)
        B_vectors[i-1, 1] = lambda_ / np.sqrt(alpha_y * epsilon[i-1] + beta_y)  # By (imaginary)
        B_vectors[i-1, 2] = bz  # Bz (real)

    # Calculate Gamma coefficients (PT-symmetric: Γz real, Γx and Γy imaginary)
    Gamma = np.zeros((N, N, 3), dtype=complex)
    for i in range(1, N+1):  # Spin indices now run from 1 to N
        for j in range(1, N+1):
            if i != j:
                denom = epsilon[i-1] - epsilon[j-1]
                Gamma[i-1, j-1, 0] = 1j*g * np.sqrt((alpha_x * epsilon[i-1] + beta_x) *
                                                     (alpha_y * epsilon[j-1] + beta_y)) / denom  # Γx imaginary
                Gamma[i-1, j-1, 1] = 1j* g * np.sqrt((alpha_y * epsilon[i-1] + beta_y) *
                                                     (alpha_x * epsilon[j-1] + beta_x)) / denom  # Γy imaginary
                Gamma[i-1, j-1, 2] = g * np.sqrt((alpha_x * epsilon[j-1] + beta_x) *
                                                (alpha_y * epsilon[j-1] + beta_y)) / denom  # Γz real

    return B_vectors, Gamma, epsilon

def construct_Qi(i, B_vectors, Gamma, N=4):
    """
    Construct PT-symmetric conserved charge Q_i
    """
    dim = 2**N

    # Initialize Q_i matrix
    Qi = np.zeros((dim, dim), dtype=complex)

    # Single spin term: B_i · σ_i
    Bi_sigma = (B_vectors[i-1, 0] * sigma_x +
                B_vectors[i-1, 1] * sigma_y +
                B_vectors[i-1, 2] * sigma_z)
    operators = [np.eye(2, dtype=complex)] * N
    operators[i-1] = Bi_sigma  # Adjusted for spin index 1 to N
    Qi_term = operators[0]
    for k in range(1, N):
        Qi_term = np.kron(Qi_term, operators[k])
    Qi += Qi_term

    # Interaction terms: ∑_{k≠i} Γ^α_ik σ^α_i σ^α_k
    for k in range(1, N+1):  # Spin indices now run from 1 to N
        if k != i:
            for alpha in range(3):
                if Gamma[i-1, k-1, alpha] != 0:
                    operators = [np.eye(2, dtype=complex)] * N
                    operators[i-1] = sigma[alpha]
                    operators[k-1] = sigma[alpha]
                    Qi_term = operators[0]
                    for j in range(1, N):
                        Qi_term = np.kron(Qi_term, operators[j])
                    Qi += Gamma[i-1, k-1, alpha] * Qi_term

    return Qi

def plot_eigenvalues_Qi(i=1):
    """
    Calculate and plot eigenvalues of PT-symmetric Q_i
    """
    B_vectors, Gamma, epsilon = define_parameters()

    # Construct Q_i
    Qi = construct_Qi(i, B_vectors, Gamma)

    # Calculate eigenvalues
    eigenvalues = np.linalg.eigvals(Qi)
    real_parts = np.real(eigenvalues)
    imag_parts = np.imag(eigenvalues)

    # Check PT symmetry (optional: verify numerically)
    Qi_conj = np.conjugate(Qi)  # Time reversal effect
    # For PT symmetry, Qi should equal its PT conjugate up to index swapping,
    # but we focus on eigenvalue reality here

    # Print parameters and eigenvalues
    print(f"Parameters for Q_{i}:")
    print(f"B_{i} = [{B_vectors[i-1,0]:.4f}, {B_vectors[i-1,1]:.4f}, {B_vectors[i-1,2]:.4f}]")
    print(f"Epsilon values: {epsilon}")
    print(f"Sample Gamma values (i={i}):")
    for k in range(1, N+1):
        if k != i:
            print(f"Γ_{i}{k} = [{Gamma[i-1,k-1,0]:.4f}, {Gamma[i-1,k-1,1]:.4f}, {Gamma[i-1,k-1,2]:.4f}]")
    print("\nEigenvalues (Real + Imaginary):")
    for n, eig in enumerate(sorted(eigenvalues, key=np.real)):
        print(f"E_{n+1} = {eig.real:.6f} + {eig.imag:.6f}i")

    # Create scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(real_parts, imag_parts, color='blue', alpha=0.6)
    plt.grid(True)
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
    plt.xlabel('$\mathrm{Re}(q_{i})$')
    plt.ylabel('$\mathrm{Im}(q_{i})$')
    plt.title(f'Eigenvalues of $Q_{i}$ ')
    plt.show()

if __name__ == "__main__":
    N = 4  # Number of spins
    # Plot eigenvalues for Q_1
    plot_eigenvalues_Qi(i=4)